# TheMovieDB.org
   https://www.themoviedb.org
   
   https://www.themoviedb.org/documentation/api?language=en-US
   
   https://developers.themoviedb.org/3/getting-started/introduction
  
### Request Limits
    40 request every 10 seconds
    https://developers.themoviedb.org/3/getting-started/request-rate-limiting

In [1]:
# Configure Environment 
from  tst.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())


Platform:  dev
Data directory :  /home/jovyan/dev/data


In [2]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import pandas as pd

import logging
logger = logging.getLogger('actorInfo')
logger.setLevel(logging.INFO)


conn = http.client.HTTPSConnection("api.themoviedb.org")
source = 'themoviedb'
payload = "{}"

apikey = 'd1fe56bd9e9956f57bb056a11cdbeb6d'

In [3]:
def searchActor(actor_name):
    urlactor = urllib.parse.quote(actor_name)
    conn.request("GET", "/3/search/person?include_adult=false&page=1&query={0}&language=en-US&api_key={1}".format(urlactor, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    for result in data['results']:
        if result['name']== actor_name:
            # return the actor ID
            actorID = result['id']
            return actorID
        else:
            print("No actor found : ", data)

In [4]:
def getActorBio(actorID):
    conn.request("GET", "/3/person/{0}?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data

In [5]:
def getActorBiometric(actorId):
    conn.request("GET", "/3/person/{0}/images?api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data['profiles']


In [6]:
def getActorRoles(actorID):
    conn.request("GET", "/3/person/{0}/movie_credits?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    cast=[]
    for role in data['cast']:
        try:
            release_date=  int( datetime.strptime(role['release_date'].strip(), '%Y-%m-%d').strftime('%Y%m%d'))
            if release_date>=20090601 and release_date <= 20190601:
                cast.append(role)
        except Exception as e:
            logger.error('ERROR parsing release date: '+ str(e))
            
    return cast


In [7]:
def getActorExternalIDs(actorID):
    conn.request("GET", "/3/person/{0}/external_ids?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data

In [8]:
def genActorMovieVertex(_actorID, actorFullName, roles):
    actor_movies_vertex =[]
    for movie in roles:
        link = {"actorID" : actorID, 
                "movieID": movie['id'],
                'characterID': movie['credit_id'],
                'character': movie['character'] , 
                'actor': actor,
                'movieTitle': movie['title'],
                '_actorID': _actorID, 
                '_source': source, 
                'actor': actorFullName, 
                'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} 
        actor_movies_vertex.append(link)
    return actor_movies_vertex


In [9]:
topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

topActors.head(15)


,Rank,First Name,Last Name,Domestic Box Office,Movies,Average
0,1,Robert,Downey Jr.,"$5,374,149,735",43,"$124,980,226"
1,2,Samuel L.,Jackson,"$5,234,777,328",60,"$87,246,289"
2,3,Scarlett,Johansson,"$4,859,403,632",28,"$173,550,130"
3,4,Tom,Hanks,"$4,692,293,839",51,"$92,005,762"
4,5,Bradley,Cooper,"$4,263,310,067",26,"$163,973,464"
5,6,Tom,Cruise,"$3,937,740,350",42,"$93,755,723"
6,7,Zoe,Saldana,"$3,932,967,185",26,"$151,267,969"
7,8,Chris,Hemsworth,"$3,915,069,280",22,"$177,957,695"
8,9,Chris,Pratt,"$3,866,023,741",17,"$227,413,161"
9,10,Harrison,Ford,"$3,850,910,074",40,"$96,272,752"


In [10]:
actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)
actors = actorsDF.values.tolist()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
import http.client



conn = http.client.HTTPSConnection("api.themoviedb.org")
source = 'themoviedb'
payload = "{}"

apikey = 'd1fe56bd9e9956f57bb056a11cdbeb6d'

for actor in actors:
    try:
        _actorID= actor[0]
        actorFullName = actor[1]
        print(actorFullName)
        actorID = searchActor(actorFullName)
        actorBio = getActorBio(actorID)
        actorBio.update({'images': getActorBiometric(actorID)} )
        actorExternalID = getActorExternalIDs(actorID)

        actorCast = getActorRoles(actorID)
        actorBio.update({'cast': actorCast})
        actorBio.update({'_actorID': _actorID, '_source': source, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} )

        f = open('./person/themoviedb/themoviedb_p_'+_actorID+'.json', 'w+')
        f.write(json.dumps(actorBio))
        f.close()

        actorMovieVertex = genActorMovieVertex( _actorID, actorFullName, actorCast)
        f2 = open('./cast/themoviedb/themoviedb_c_'+_actorID+'.json', 'w+')
        f2.write(json.dumps(actorMovieVertex))
        f2.close()
        print('Processed : ', actor)
        time.sleep(15)
    except Exception as e:
        logger.error('ERROR processing actor bio: '+ str(e))


Robert Downey Jr.


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Processed :  ['1', 'Robert Downey Jr.']


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Samuel L. Jackson
Processed :  ['2', 'Samuel L. Jackson']


...

Chris Pine
Processed :  ['96', 'Chris Pine']
Jon Favreau
Processed :  ['97', 'Jon Favreau']


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Nicole Kidman
Processed :  ['98', 'Nicole Kidman']


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Simon Pegg
Processed :  ['99', 'Simon Pegg']
Taylor Lautner
Processed :  ['100', 'Taylor Lautner']


# Download profile images and label them into a subfolder for each actor


In [15]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import urllib
import http.client
import json
import pandas as pd
conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"

for actor in actors:
    try:
        _actorID= actor[0]
        actorFullName = actor[1]

        actorID = searchActor(actorFullName)
        images = getActorBiometric(actorID)
        images
        baseURL='https://image.tmdb.org/t/p/original/'
        prefix='themoviedb_i_'+_actorID+'_'

        if not os.path.exists('./person/images/'+_actorID):
            os.makedirs('./person/images/'+_actorID, exist_ok=True)

        for image in images:
            image_name = image['file_path'][1:]
            image_output = prefix+image_name
            f = open('./person/images/'+_actorID+'/'+image_output,'wb')
            f.write(urllib.request.urlopen(baseURL+image_name).read())
            f.close()
    except Exception as e:
        logger.error('ERROR processing actor bio: '+ str(e))

In [18]:
# Copy models up to S3 bucket
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./person/ s3://' + bucket.strip() + '/person/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

bdso-celebrity-content-dev-vpc-05447d97de18e28f8

aws s3 sync ./person/ s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/person/ 
Completed 88.6 KiB/~2.7 MiB (2.1 MiB/s) with ~12 file(s) remaining (calculating...)
upload: person/images/1/themoviedb_i_1_1YjdSym1jTG7xjHSI0yGGWEsw5i.jpg to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/person/images/1/themoviedb_i_1_1YjdSym1jTG7xjHSI0yGGWEsw5i.jpg
Completed 88.6 KiB/~2.7 MiB (2.1 MiB/s) with ~11 file(s) remaining (calculating...)
Completed 344.6 KiB/~3.2 MiB (5.1 MiB/s) with ~16 file(s) remaining (calculating...)
Completed 600.6 KiB/~4.2 MiB (6.0 MiB/s) with ~24 file(s) remaining (calculating...)
Completed 737.1 KiB/~5.8 MiB (6.0 MiB/s) with ~37 file(s) remaining (calculating...)
upload: person/images/1/themoviedb_i_1_5qHNjhtjMD4YWH3UP0rm4tKwxCL.jpg to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/person/images/1/themoviedb_i_1_5qHNjhtjMD4YWH3UP0rm4tKwxCL.jpg
Completed 737.1 KiB/~5.8 MiB (6.0 MiB/s) with ~36 file(s) re